In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('ggplot')

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import RandomizedSearchCV

from keras.models import Sequential
from keras import layers
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd

In [3]:
filepath_dict = {
    'yelp' : '../input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/yelp_labelled.txt',
    'amazon' : '../input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/amazon_cells_labelled.txt',
    'imdb' : '../input/sentiment-labelled-sentences-data-set/sentiment labelled sentences/imdb_labelled.txt'
}

In [4]:
df_list = []

for src, fp in filepath_dict.items():
    df = pd.read_csv(fp, names=['sentence', 'label'], sep='\t')
    df['source'] = src
    df_list.append(df)

In [5]:
df = pd.concat(df_list)
df.iloc[0]

sentence    Wow... Loved this place.
label                              1
source                          yelp
Name: 0, dtype: object

In [6]:
df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [7]:
df_yelp = df[df['source'] == 'yelp']
sentences = df_yelp['sentence'].values
y = df_yelp['label'].values



In [8]:
sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, test_size=0.2)

In [11]:
vectorizer = CountVectorizer()
vectorizer.fit(sentences_train)

CountVectorizer()

In [14]:
x_train = vectorizer.transform(sentences_train)
x_test = vectorizer.transform(sentences_test)

In [18]:
classifier = LogisticRegression()
classifier.fit(x_train, y_train)
score = classifier.score(x_test, y_test)

In [19]:
score

0.83

In [30]:
# DNN
# input_dim = x_train.shape[1]

model = Sequential([
    layers.Dense(4, input_dim=x_train.shape[1], activation='relu'),
    layers.Dropout(0.05),
    layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [31]:
model.fit(x_train, y_train, epochs=100, validation_data=(x_test,y_test), batch_size=16)

Epoch 1/100
50/50 [==============================] - 0s 3ms/step - loss: 0.6887 - accuracy: 0.5412 - val_loss: 0.6799 - val_accuracy: 0.6650
Epoch 2/100
50/50 [==============================] - 0s 2ms/step - loss: 0.6673 - accuracy: 0.6850 - val_loss: 0.6607 - val_accuracy: 0.7300
Epoch 3/100
50/50 [==============================] - 0s 2ms/step - loss: 0.6393 - accuracy: 0.7613 - val_loss: 0.6389 - val_accuracy: 0.7600
Epoch 4/100
50/50 [==============================] - 0s 2ms/step - loss: 0.6082 - accuracy: 0.8050 - val_loss: 0.6164 - val_accuracy: 0.7750
Epoch 5/100
50/50 [==============================] - 0s 2ms/step - loss: 0.5736 - accuracy: 0.8313 - val_loss: 0.5934 - val_accuracy: 0.7950
Epoch 6/100
50/50 [==============================] - 0s 2ms/step - loss: 0.5394 - accuracy: 0.8562 - val_loss: 0.5707 - val_accuracy: 0.8000
Epoch 7/100
50/50 [==============================] - 0s 2ms/step - loss: 0.5053 - accuracy: 0.8813 - val_loss: 0.5476 - val_accuracy: 0.8250
Epoch 8/100
5

Epoch 59/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0363 - accuracy: 0.9937 - val_loss: 0.6139 - val_accuracy: 0.7950
Epoch 60/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0290 - accuracy: 0.9962 - val_loss: 0.6226 - val_accuracy: 0.8000
Epoch 61/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0363 - accuracy: 0.9925 - val_loss: 0.6298 - val_accuracy: 0.8000
Epoch 62/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0369 - accuracy: 0.9925 - val_loss: 0.6392 - val_accuracy: 0.8000
Epoch 63/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0289 - accuracy: 0.9962 - val_loss: 0.6475 - val_accuracy: 0.8000
Epoch 64/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0282 - accuracy: 0.9975 - val_loss: 0.6615 - val_accuracy: 0.8050
Epoch 65/100
50/50 [==============================] - 0s 2ms/step - loss: 0.0307 - accuracy: 0.9950 - val_loss: 0.6712 - val_accuracy: 0.8050
Epoch 

In [32]:
model.evaluate(x_test, y_test)

7/7 [==============================] - 0s 1ms/step - loss: 0.9918 - accuracy: 0.8050


[0.9918497204780579, 0.8050000071525574]

In [35]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(sentences_train)

x_train = tokenizer.texts_to_sequences(sentences_train)
x_test = tokenizer.texts_to_sequences(sentences_test)

In [36]:
x_train = pad_sequences(x_train, maxlen=100)
x_test = pad_sequences(x_test, maxlen=100)

In [52]:
model2 = Sequential([
    layers.Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=50, input_length=100),
    layers.Conv1D(128, 5, activation='relu'),
    layers.GlobalMaxPool1D(),
    layers.Dense(10, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])
model2.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [53]:
model2.fit(x_train, y_train, epochs=100, validation_data=(x_test,y_test), batch_size=16)

Epoch 1/100
50/50 [==============================] - 1s 11ms/step - loss: 0.6922 - accuracy: 0.4863 - val_loss: 0.6815 - val_accuracy: 0.5950
Epoch 2/100
50/50 [==============================] - 0s 8ms/step - loss: 0.6564 - accuracy: 0.5800 - val_loss: 0.6357 - val_accuracy: 0.7300
Epoch 3/100
50/50 [==============================] - 0s 9ms/step - loss: 0.5780 - accuracy: 0.7325 - val_loss: 0.5828 - val_accuracy: 0.6500
Epoch 4/100
50/50 [==============================] - 0s 8ms/step - loss: 0.4855 - accuracy: 0.8450 - val_loss: 0.5413 - val_accuracy: 0.7550
Epoch 5/100
50/50 [==============================] - 0s 8ms/step - loss: 0.4177 - accuracy: 0.8975 - val_loss: 0.5606 - val_accuracy: 0.7300
Epoch 6/100
50/50 [==============================] - 0s 8ms/step - loss: 0.3492 - accuracy: 0.9425 - val_loss: 0.5015 - val_accuracy: 0.7700
Epoch 7/100
50/50 [==============================] - 0s 8ms/step - loss: 0.1956 - accuracy: 0.9663 - val_loss: 0.4159 - val_accuracy: 0.8200
Epoch 8/100


Epoch 59/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0103 - accuracy: 0.9950 - val_loss: 3.4345 - val_accuracy: 0.7700
Epoch 60/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0077 - accuracy: 0.9987 - val_loss: 3.4858 - val_accuracy: 0.7700
Epoch 61/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0106 - accuracy: 0.9962 - val_loss: 3.5155 - val_accuracy: 0.7650
Epoch 62/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0101 - accuracy: 0.9975 - val_loss: 3.7409 - val_accuracy: 0.7750
Epoch 63/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0047 - accuracy: 1.0000 - val_loss: 3.7795 - val_accuracy: 0.7750
Epoch 64/100
50/50 [==============================] - 0s 8ms/step - loss: 0.0106 - accuracy: 0.9975 - val_loss: 3.8375 - val_accuracy: 0.7750
Epoch 65/100
50/50 [==============================] - 1s 10ms/step - loss: 0.0071 - accuracy: 0.9987 - val_loss: 3.8263 - val_accuracy: 0.7750
Epoch

In [54]:
model2.evaluate(x_test, y_test)

7/7 [==============================] - 0s 3ms/step - loss: 4.2129 - accuracy: 0.7750


[4.212886333465576, 0.7749999761581421]